### UPDATE

Our team won the first place prize in the class competition! 

Read about it here: https://uwaterloo.ca/school-of-accounting-and-finance/news/congratulations-teams-won-cfm-101-team-competition.

---

### Group Assignment
### Team Number: 3
### Team Member Names: Derek Tan, Jeff Peng, Yuqian Lin
### Team Strategy Chosen: SAFE

### Abstract

Our portfolio optimization strategy involves the use and implementation of the Modern Portfolio Theory (MPT) and analysis of the Efficient Frontier graphs. The objective of the portfolio optimization strategy is to maximize the portfolio return while maintaining the minimum portfolio risk. 

Modern Portfolio Theory states that since it is assumed that all investors are risk-adverse, when considering the possible portfolio allocation strategies, the investor will prefer the portfolio that maximizes the possible return while maintaining a given amount of risk. 

The Efficient Frontier (EF), the core of our strategy, was introduced by Nobel Laureate Harry Markowitz and is fundamental to MPT. The EF is a graph that illustrates all possible portfolios portfolio allocation distributions. The x-axis represents the volatility/risk of the portfolio, while the y-axis represents the expected return of the portfolio.

The Efficient Frontier shows the optimized portfolios that offer the highest expected return for a given level of risk and the lowest level of risk for a given level of expected return.

An example of the an Efficient Frontier graph is shown below:

![EF Graph](ef.png)

As seen from the graph, the light blue dot is the portfolio that takes on the highest level of risk coupled with the highest degree of return. Conversely, the left-most purple dot depicts the portfolio that with the lowest level of risk and lowest given level of return. Typically, risk-seeking investors will select portfolios that lie on the right end as they yield a higher return for a high level of risk. In our group's case, we chose the "safe" strategy, and thus will be selecting the portfolio on the left-end of the graph as it yields a lower return for a lower level of risk.

Note: We will be discussing more about how we graphed each portfolio along the EF graph below. Additionally, we will also provide a comprehensive conclusion of our strategy and analysis at the very end.

In [ ]:
from IPython.display import display, Math, Latex
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
#Imports CSV file's tickers a list
ticker_df = pd.read_csv("Tickers.csv", index_col=False)
results = list(ticker_df.iloc[:, 0])
results.insert(0, ticker_df.columns[0])

In [ ]:
# filters_nan(ticker_list) filters out stocks with invalid prices, that is stocks with NaN values.
# It then produces a list of these filtered tickers

"""
Params:
    ticker_list (listof Str): Unfiltered original list of tickers
"""


def filters_nan(ticker_list):
    start_date = "2018-01-02"
    end_date = "2021-10-31"
    no_nan = []

    for i in range(ticker_list.shape[1]):
        if ticker_list.iloc[:, i].isna().sum() < len(ticker_list) - 209:
            no_nan.append((ticker_list.columns)[i])

    return no_nan

In [ ]:
# Import Financial Data

unfiltered_tickers = results

start_date = "2018-01-02"
end_date = "2021-10-31"

data = yf.download(unfiltered_tickers, start=start_date, end=end_date)

data

In [ ]:
# Get Closing Prices and Volume Data
closing_data = data["Adj Close"]
volume_data = data["Volume"]

volume_start_date = datetime.strptime("2021-07-02", "%Y-%m-%d")
volume_end_date = datetime.strptime("2021-10-22", "%Y-%m-%d")

# Filter based on Volume
volume = volume_data.loc[volume_start_date:volume_end_date].mean().dropna()

volume2 = pd.DataFrame(volume, columns=["Volume"])

volume_valid = list(volume2[volume2.Volume > 10000].index)

closing_data = closing_data[volume_valid]

# Filter based on NaN values

no_nan_tickers = filters_nan(closing_data)

no_nan = closing_data[no_nan_tickers]

# Filter based on currency

valid_tickers = []

for i in range(len(no_nan_tickers)):
    info = yf.Ticker(no_nan_tickers[i]).info
    div = info.get("currency")
    if div == "USD":
        valid_tickers.append(no_nan_tickers[i])

In [ ]:
closing_prices = closing_data[valid_tickers]

closing_prices

To compare price fluctuations, we will calculate the daily percentage change in the price of each stock. By calculating percent change, it makes it easier to compare price fluctuations between stocks as the magnitude of the price changes will be compared, removing the influence of the share price from the price fluctuations.

In [ ]:
# Calculate percent change

percent_change = closing_prices.pct_change().apply(lambda x: np.log(1 + x))

percent_change.head()

### Constructing the Efficient Frontier Graph
To construct an Efficient Frontier Graph, we require three factors:
- Covariance of the securities in the portfolio
- Standard deviation also known as risk
- The expected return of the portfolio

Below, we will be calculating all these three factors.

#### Covariance

We will now analyze the covariance of each stock in relation to one another. The covariance of two stocks (stock X, stock Y) is calculated using the following equation:

\begin{align*}
COV(X,Y)=\frac{\sum(x_i-\overline{X})\times(y_i-\overline{Y})}{N}
\end{align*}

We will store the results of the covariance calculations in 'cov_matrix'.

In [ ]:
cov_matrix = percent_change.cov()

cov_matrix

#### Standard Deviation

To calculate standard deviation, we need to calculate the correlation between stocks.

To do this, we will use a correlation matrix.

The correlation of two stocks (stock X, stock Y) is calculated using the following equation:

\begin{align*}
\rho(X,Y)=\frac{COV(X,Y)}{\sigma_X \sigma_Y}
\end{align*}

Where $\rho_{x,y}$ is the correlation between the two variables, $cov(r_x, r_y)$ is the covariance of return X and return Y, and $\sigma_x$ and $\sigma_y$ are the standard deviations of X and Y respectively.

Note that each stock has a correlation of 1 with itself, a perfect positive correlation.

There exists a positive correlation between stocks X and Y if $0 < \rho_{x,y} < 1$.

There exists a negative (inverse) correlation between stocks X and Y if $-1 < \rho_{x,y} < 0$.

There exists no (zero) correlation between stocks X and Y if $\rho_{x,y} = 0$. In reality, it is almost impossible for two stocks to have zero correlation with each other.

We will store the results of the correlation calculations in 'corr_matrix'.

In [ ]:
corr_matrix = percent_change.corr()

corr_matrix

#### Expected Return
Finally, we will calculate the expected return of each portfolio. The expected return of a portfolio is caluclated by the equation below:

\begin{align*}
E(X)=\overline{X}=\frac{\sum x_i}{N}
\end{align*}

where $x_i$ are individual returns of some security $X$, $N$ is the total number of observations (time periods for us)

**Volatility**

We now want to calculate the yearly volatility rate of each stock (which is equivalent to the annual standard deviation). Through this calculation, we know which stocks have the highest price fluctuations, or essentially, which stocks are the riskiest.

In [ ]:
# Calculate Yearly Expected Returns (Returns)

closing_prices.index = pd.to_datetime(closing_prices.index)

individual_expected_returns = closing_prices.resample("Y").first().pct_change().mean()

yearly_stats = pd.DataFrame(individual_expected_returns, columns=["Returns"])

# Calculate Annual Standard Deviation (Volatility)

trading_days = 250

annual_standard_deviation = percent_change.std().apply(
    lambda x: x * np.sqrt(trading_days)
)

yearly_stats["Volatility"] = annual_standard_deviation

yearly_stats

Now we sort the stocks and their annual volatility rates in increasing order and take the first twenty tickers if there are 20 or more tickers total, otherwise we use all the stocks. We do this because we want the stocks with the lowest volatility rates so that we can create the least risky portfolio.

In [ ]:
sorted_volatility = yearly_stats.sort_values("Volatility")

lowest_volatility_tickers = list(
    sorted_volatility["Volatility"][
        0 : (20 if len(yearly_stats) > 20 else len(yearly_stats))
    ].index
)

yearly_stats = yearly_stats.loc[lowest_volatility_tickers]

cov_matrix = cov_matrix.loc[lowest_volatility_tickers][lowest_volatility_tickers]

Now we generate 10000 random portfolios from the 20 tickers we chose (with lowest volatility). We then find the portfolio with the lowest volatility (the safest portfolio) and find the weightings for each stock that created the portfolio.

Note: You can change this number to change the number of randomly generated portfolios. The more number of random portfolios generated, the more optimized the final optimized portfolio will be.

In [ ]:
number_of_portfolios = 10000

In [ ]:
# Generate portfolios with random weights

# generate_portfolios(tickers, number_of_portfolios) generates
#   a collection of [number_of_portfolios] portfolios from the
#   list of [tickers]

"""
Params:
    tickers (listof Str): List of stock tickers to choose from
    number_of_portfolios (Nat): Number of portfolios to generate
"""


def generate_portfolios(tickers, number_of_portfolios):
    weights = []
    returns = []
    volatility = []

    for i in range(number_of_portfolios):
        individual_weights = np.random.random(len(tickers))
        individual_weights = individual_weights / np.sum(individual_weights)
        weights.append(individual_weights)

        individual_returns = np.dot(individual_weights, yearly_stats.Returns)
        returns.append(individual_returns)

        portfolio_variance = (
            cov_matrix.mul(individual_weights, axis=0)
            .mul(individual_weights, axis=1)
            .sum()
            .sum()
        )
        standard_deviation = np.sqrt(portfolio_variance)
        individual_volatility = standard_deviation * np.sqrt(trading_days)
        volatility.append(individual_volatility)

    portfolios = pd.DataFrame(index=range(number_of_portfolios))

    portfolios["Returns"] = returns
    portfolios["Volatility"] = volatility

    for i in range(len(tickers)):
        for j in range(number_of_portfolios):
            portfolios[tickers[i]] = weights[j][i]

    return portfolios


random_portfolios = generate_portfolios(lowest_volatility_tickers, number_of_portfolios)

random_portfolios

In [ ]:
# Pick optimal portfolio with individual stock weighting

safest_portfolio = random_portfolios.iloc[random_portfolios.Volatility.idxmin()]

pd.DataFrame(safest_portfolio)

In [ ]:
# Plot Efficient Frontier Graph

plt.subplots(figsize=[10, 10])

plt.scatter(
    x=random_portfolios.Volatility, y=random_portfolios.Returns, s=10, alpha=0.7
)

plt.scatter(
    safest_portfolio.Volatility, safest_portfolio.Returns, color="r", marker="*", s=200
)

plt.title("Efficient Frontier of Randomly Generated Portfolios")
plt.xlabel("Volatility")
plt.ylabel("Returns")

plt.show()

In [ ]:
# Replace date with Nov. 26, 2021 (format: 'YYYY-MM-DD')

date = "2021-11-26"

In [ ]:
# Produce final list of chosen tickers and weights

initial_capital = 100000

current_day = datetime.strptime(date, "%Y-%m-%d")

next_day = current_day + timedelta(days=1)

final_portfolio_columns = ["Ticker", "Price", "Shares", "Value", "Weight"]

FinalPortfolio = pd.DataFrame(columns=final_portfolio_columns)

safest_portfolio_data = safest_portfolio[2:]

safest_portfolio_data = safest_portfolio_data.sort_index()

safest_portfolio_tickers = list(safest_portfolio_data.index)

safest_portfolio_weights = list(safest_portfolio_data.values)

prices = yf.download(safest_portfolio_tickers, start=current_day, end=next_day)

In [ ]:
current_prices = prices["Adj Close"].loc[date]

FinalPortfolio['Ticker'] = safest_portfolio_tickers

FinalPortfolio['Price'] = current_prices.values

FinalPortfolio['Weight'] = safest_portfolio_weights

FinalPortfolio['Value'] = initial_capital * FinalPortfolio.Weight

FinalPortfolio['Shares'] = FinalPortfolio.Value / FinalPortfolio.Price

FinalPortfolio.index = list(range(1, 21))

FinalPortfolio

In [ ]:
# Sanity Check

total_portfolio_value = FinalPortfolio.Value.sum().round(2)

if total_portfolio_value == initial_capital:
    print(f"The portfolio's total value is ${initial_capital}")

total_weight = FinalPortfolio.Weight.sum().round(2)

if total_weight == 1:
    print("The portfolio's weights add up to 100%.")
    


In [ ]:
# Export FinalPortfolio Data to CSV

Stocks = FinalPortfolio[['Ticker', 'Shares']]

Stocks.to_csv("Stocks_Group_3.csv", encoding='utf-8')

### Conclusion and Data Analysis ###

Our strategy was to generate a list of the top 20 least volatile stocks from the beginning of 2018 to late 2021. The reason why we chose the maximum number of 20 stocks is because, as we learned in class, splitting a portfolio over more stocks reduces the portfolio's risk.  

After constructing our Efficient Frontier graph, with 10000 possible portfolios, our program selects the portfolio with the lowest volatility (the portfolio closest to the left). Since our strategy is to be safe (ie. generate a portfolio that gives us an "ending value" closest to zero), we decided to choose the portfolio with the lowest volatility because that should theoretically be the least risky and exported it as a CSV file.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Derek, Yuqian, Jeff

### Sources

Image Link: https://www.cryptimi.com/guides/is-diversification-the-right-strategy-for-your-cryptocurrency-portfolio

Equations: Professor Thompson's notes


Definition of MPT & EF: https://www.investopedia.com/terms/e/efficientfrontier.asp https://www.investopedia.com/terms/m/modernportfoliotheory.asp
